In [1]:
import nltk
import spacy

nltk.download('punkt')
nltk.download('rslp')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/igorecarvalho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     /home/igorecarvalho/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [2]:
#importação das bibliotecas criadas para normalização dos dados
from nlputils import lexical
from nlputils import morphosyntax
from nlputils import syntax

In [3]:
#chamada da bibioteca de preprocessamento
lexical_normalizer = lexical.Preprocessing()
morphosyntax_normalizer = morphosyntax.Preprocessing('../models/pt_core_news_sm-2.1.0')
syntax_normalizer = syntax.Preprocessing('../models/pt_core_news_sm-2.1.0')

In [4]:
import os

#definição do diretorio dos corpus e criacao de uma lista com os nomes de cada arquivo dentro do diretorio
corpora_path = '../data/corpora/'
files_corpora = os.listdir(corpora_path)
files_corpora = [d for d in files_corpora if d not in '.DS_Store']

In [5]:
#criacao de um dicionario que ira armazenar cada corpus em uma chave
sentences_dic = {}
all_files = []
for corpus in files_corpora:
    files = [os.path.join(corpora_path + corpus, f) \
             for f in os.listdir(corpora_path + corpus) \
             if os.path.isfile(os.path.join(corpora_path + corpus, f))]
    #cada corpus tera mais 4 chaves para armazenar informacoes de trabalho. Obs.: svo = sujeito, verbo, objeto
    sentences_dic[corpus] = {'sentencas': [], 'tag': [], 'parse': [], 'svo': []}
    
    #adiciona todos os arquivos em uma unica lista independentemente do corpus
    print(len(files))
    all_files.extend(files)
    
    #para cada arquivo em um corpus sera extraido suas frases e armazenadas em cada linha de uma lista
    for file in files[:30]:
        with open(file, 'r', encoding='utf-8') as text_file:
            lines = text_file.readlines()
            for line in lines:
                if line != '\n':
                    #toda a sentenca sera escrita em letras minusculas
                    #line = lexical_normalizer.lowercase(line) 
                    #tokeniza as sentencas
                    sentences_line = lexical_normalizer.tokenize_sentences(line)
                    for sentence in sentences_line:
                        #print(sentence)
                        #adiciona cada sentenca de uma linha no dicionario
                        sentences_dic[corpus]['sentencas'].append(sentence)
                        #adiciona uma lista de cada palavra da sentenca taggeada composta de 
                        #(token, etiqueta_morfossintática)
                        sentences_dic[corpus]['tag'].append(morphosyntax_normalizer.tag(sentence))
                        #adiciona uma lista de cada palavra da sentenca composta de 
                        #(token, papel_sintático, head),
                        sentences_dic[corpus]['parse'].append(syntax_normalizer.parse(sentence))
                        #adiciona uma lista de cada sentenca composta por tuplas de (sujeito, verbo, objeto)
                        sentences_dic[corpus]['svo'].append(syntax_normalizer.get_SVO(sentence))
                    

538
664


In [8]:
sentences_dic.keys()

dict_keys(['animais', 'games'])

In [9]:
#utilizacao do Pandas para visualizao dos dados em forma de tabelas
import pandas as pd

#cracao de um dicionario que ira armazenar cada corpus em suas respectivas keys.
dataframes_sentences = {}
for key in sentences_dic.keys():
    #os corpus armazenados aqui estara em formato de DataFrame onde cada key sera uma coluna da tabela
    dataframes_sentences[key] = pd.DataFrame(sentences_dic[key], columns=['sentencas','tag','parse', 'svo'])

In [10]:
dataframes_sentences['animais'].head(n=5)

,sentencas,tag,parse,svo
0,Dizem que fomos criados para amar e sermos ama...,"[(Dizem, VERB), (que, SCONJ), (fomos, ADJ), (c...","[(Dizem, ROOT, Dizem), (que, mark, criados), (...",[]
1,"Por isso, quando nos privam do amor ou da opor...","[(Por, ADP), (isso, PRON), (,, PUNCT), (quando...","[(Por, case, isso), (isso, obl, sentir), (,, p...",[]
2,"Este foi o caso de Tonka, uma Vombate australi...","[(Este, PRON), (foi, VERB), (o, DET), (caso, N...","[(Este, nsubj, caso), (foi, cop, caso), (o, de...","[(Este, foi, caso), (que, teve, sérios), (Você..."
3,Contaremos um pouco sobre este animal.Caracter...,"[(Contaremos, VERB), (um, DET), (pouco, DET), ...","[(Contaremos, ROOT, Contaremos), (um, det, pou...","[(Vombate, é, estranho)]"
4,Têm a aparência de um urso musculoso de tamanh...,"[(Têm, VERB), (a, DET), (aparência, NOUN), (de...","[(Têm, ROOT, Têm), (a, det, aparência), (aparê...","[(que, permitem, grandes)]"


In [11]:
dataframes_sentences['games'].head(n=5)

,sentencas,tag,parse,svo
0,Historiador carioca que trabalha como Redator ...,"[(Historiador, NOUN), (carioca, VERB), (que, P...","[(Historiador, nsubj, há), (carioca, amod, His...","[(Historiador, há, carioca), (que, trabalha, R..."
1,Já se aventurou mais de uma vez pelo mundo da ...,"[(Já, ADV), (se, PRON), (aventurou, VERB), (ma...","[(Já, advmod, aventurou), (se, expl, aventurou...",[]
2,Apesar de escrever sobre todo tipo de coisa pa...,"[(Apesar, ADV), (de, ADP), (escrever, VERB), (...","[(Apesar, advmod, tem), (de, mark, escrever), ...",[]
3,"Como jogar Clash Royale no PC, o game de carta...","[(Como, ADV), (jogar, VERB), (Clash, PROPN), (...","[(Como, advmod, jogar), (jogar, ROOT, jogar), ...","[(Clash, jogar, game), (Royale, é, jogo)]"
4,Apesar da exclusividade para plataformas mobil...,"[(Apesar, ADV), (da, ADP), (exclusividade, NOU...","[(Apesar, advmod, levar), (da, obl, Apesar), (...","[(truque, levar, da), (processo, é, gratuito)]"


> Utilizando	 o	 corpora	 compilado	 para	 a	 Prova	 1, e	 as	 rotinas	
definidas	 na	 questão	 anterior, realizar	 a	 extração	 de	 informações	 no	 formado	 de	 triplas:	
(Sujeito,	Verbo,	Objeto).


> Um	dicionário	Python	deve	ser	criado	da	seguinte	forma:
- a. {“verbo lematizado1”: [(Sujeito1, Objeto1), (Sujeito2,
None), ..., (Sujeiton, Objeton)],
...,
“verbo lematizadok”: [(Sujeito1, Objeto1), (Sujeito2,
Objeto2), ..., (Sujeitom, Objetom)]}

In [12]:
#dicionario contendo lista de verbos e para cada verbo uma lista de tuplas contendo (Sujeito, Objeto)
lemma_verb = {}
#lista de tuplas dos verbos que nao possuem sujeito no formato (verbo lematizado, verbo encontrado)
no_obj_verb = []
for corpus in sentences_dic:
    for sentence in sentences_dic[corpus]['svo']:
        for svo in sentence:
            if svo[1] != None:
                verb = svo[1].lemma_
                if verb in lemma_verb.keys():
                    lemma_verb[verb].append((svo[0], svo[2]))
                else:
                    lemma_verb[verb] = []
                    lemma_verb[verb].append((svo[0], svo[2]))
                if svo[2] == None and svo[2] not in no_obj_verb:
                    no_obj_verb.append((verb, svo[1]))

> - b. Exibir	as	seguintes	estatísticas:

In [ ]:
stat_verbs = {'verbo': [], 'num_svo': []}
for verb in lemma_verb:
    stat_verbs['verbo'].append(verb)
    stat_verbs['num_svo'].append(len(lemma_verb[verb]))

> - i. Qual	verbo	tem	a	maior	lista	de	sujeitos	e	objetos

In [ ]:
df = pd.DataFrame(stat_verbs, columns=['verbo','num_svo'])
df.sort_values(by="num_svo").tail(n=1)

> - ii.Há	algum	verbo	sem	objetos?	Mostre	alguns.

In [ ]:
print(len(no_obj_verb))
print(no_obj_verb[:10])

> Dada	uma	pergunta,	no	formato “O	que/quem	verbo?”	responde “O	que/quem	verbo	objeto”.
- Exemplos	de	perguntas:
Quem	pintou	um	quadro?
O	que	derrubou	os	preços?
- Exemplos	de	respostas:
Michelangelo	pintar	quadros.
Notícias	ruins	derrubar	os	preços.

In [19]:
question = input('Faça uma pergunta no formato:  “O que/quem verbo?” ')
ext_data = syntax_normalizer.get_SVO(question)

Insira seu login: VTINHI
‘Olá, ‘VTINHI


In [23]:
for pos_respostas in ext_data:
    verbo = pos_respostas[1].lemma_
    obj = pos_respostas[2]
    for tupla in lemma_verb[verbo]:
        if tupla[1] != None and obj.text == tupla[1].text:
            suj = tupla[0]
            print(str(suj) + ' ' + str(verbo) + ' ' + str(obj))